In [ ]:
import PyPDF2 as pypdf

In [ ]:
def findInDict(needle, haystack):
    for key in haystack.keys():
        try:
            value=haystack[key]
        except:
            continue
        if key==needle:
            return value
        if isinstance(value,dict):            
            x=findInDict(needle,value)            
            if x is not None:
                return x
pdfobject=open('../data/raw/boletim_covid/boletim-direitos-na-pandemia.pdf','rb')
pdf=pypdf.PdfFileReader(pdfobject)
xfa=findInDict('/XFA',pdf.resolvedObjects)
# xml=xfa[7].getObject().getData()

In [ ]:
def text_extractor(path):
    with open(path, 'rb') as f:
        pdf = pypdf.PdfFileReader(f)
        # get the first page
        page = pdf.getPage(7)
        print(page)
        print('Page type: {}'.format(str(type(page))))
        text = page.extractText()
        print(text)
if __name__ == '__main__':
    path = '../data/raw/boletim_covid/boletim-direitos-na-pandemia.pdf'
    text_extractor(path)

## Tika

In [ ]:
# import parser object from tika
from tika import parser

In [ ]:
# opening pdf file
parsed_pdf = parser.from_file("../data/raw/boletim_covid/CEPEDISA-USP-Linha-do-Tempo-Maio-2021_v3.pdf")

In [ ]:
covid_events = parsed_pdf['content'].split('\n\n \n\n \n\n \n\nFEVEREIRO DE 2020 \n\n \n\n \n\n')[1].split('\n\n \n\n \n\n \n\n \n\nÍNDICE REMISSIVO \n\n \n\n \n\n')[0]

In [ ]:
import re

In [ ]:
match = re.split(r'(\d+/\d+/\d+)\s(\bAtos normativos\b|\bAtos de governo\b|\bPropaganda\b)', covid_events)

In [ ]:
match

In [ ]:
MARÇO DE 2020 \n\n \n\n \n\n \n\n \n\n
ABRIL DE 2020 \n\n \n \n\n
MAIO DE 2020 \n\n \n\n',

In [ ]:
a = ' \n\n \n\n3. Ministério da Saúde apresenta Plano de Contingência Nacional para Infecção \n\nHumana pelo novo Coronavírus que estipula os seguintes eixos de atuação: \n\nvigilância, suporte laboratorial,  medidas de controle de infecção,   assistência \n\nfarmacêutica, vigilância sanitária, medidas de saúde em pontos de entrada \n\n(portos, aeroportos e passagens de fronteiras), comunicação de risco e \n\ngestão30. No eixo de comunicação de risco, o referido plano detalha quatorze \n\nações voltadas à prevenção que serão iniciadas mas abandonadas nos meses \n\nseguintes (ver §§ 76, 131, 200, 221 e 229). Em 26/02/20, é confirmado o \n\nprimeiro caso de Covid-19 no Brasil. Trata-se de um homem de 61 anos \n\ncom histórico de viagem para Itália, região da Lombardia31. \n\n \n\n \n\n                                                 \n30 Disponível em: https://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/plano-\n\ncontingencia-coronavirus-COVID19.pdf  \n31 “Coronavírus: Brasil confirma primeiro caso da doença”, UMA-SUS. Disponível em: \n\nhttps://www.unasus.gov.br/noticia/coronavirus-brasil-confirma-primeiro-caso-da-doenca  \n\nSemana epidemiológica 9, 25-29/02/20 \n\n \n\nCasos de Covid-19 acumulados – 2 \n\nÓbitos por Covid-19 acumulados – 0 \n\n \n\nhttps://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/plano-contingencia-coronavirus-COVID19.pdf\nhttps://portalarquivos2.saude.gov.br/images/pdf/2020/fevereiro/13/plano-contingencia-coronavirus-COVID19.pdf\nhttps://www.unasus.gov.br/noticia/coronavirus-brasil-confirma-primeiro-caso-da-doenca\n\n\n31 \n\n  \n \n\n \n\n \n\n \n\n \n\nMARÇO DE 2020 \n\n \n\n \n\n \n\n \n\n'

In [ ]:
event = re.split(r'\s{49}', a)
if len(event) > 1:
    event

In [ ]:
aux = 1
for i in range(1, len(match)):
    if aux == 1:
        date = match[i]
    elif aux == 2:
        event_type = match[i]
    else:
        aux = 1
#         event = re.sub(r'\n', '', match[i])
#         event = re.split(r'\s{49}', '', match[i])
        event = re.sub(r'\n\n\n(\d+) \n\n  \n \n\n \n\n \n\n \n\n \n\n', '', match[i])
#         event = re.sub(r')
        print(match[i])
        continue
    aux = aux + 1

## PDF Miner

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [ ]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            
        text = fake_file_handle.getvalue()
    
    # close open handles
    converter.close()
    fake_file_handle.close()
    
    if text:
        return text

In [ ]:
if __name__ == '__main__':
    print(extract_text_from_pdf('../data/raw/boletim_covid/boletim-direitos-na-pandemia.pdf'))

In [ ]:
import pdfminer
from pdfminer.high_level import extract_pages

In [ ]:
for page_layout in extract_pages("../data/raw/boletim_covid/boletim-direitos-na-pandemia.pdf"):
    for element in page_layout:
        print(element)

# TESTES

In [ ]:
import pandas as pd

In [ ]:
df_casos = pd.read_parquet('../data/app/covid_saude_obito_municipio.parquet')
df_casos['data'] = pd.to_datetime(df_casos['data'])

df_boletim_direitos = pd.read_csv('../data/raw/boletim_covid/boletim_direitos_covid.csv', encoding='latin-1')
df_boletim_direitos['data'] = pd.to_datetime(df_boletim_direitos['data'], format='%d/%m/%Y')

df_boletim_direitos['noticia'] = df_boletim_direitos['noticia'].str.wrap(30)
df_boletim_direitos['noticia'] = df_boletim_direitos['noticia'].apply(lambda x: x.replace('\n', '<br>'))

df_casos = df_casos.merge(df_boletim_direitos, how='left', on='data')

In [ ]:
df_casos.loc[(df_casos['municipio']=='São Paulo')&(df_casos['noticia'].notna())]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_casos = df_casos.loc[df_casos['municipio']=='São Paulo']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_casos["data"], 
                         y=df_casos["obitosNovos"], 
                         text=df_casos['noticia'],
                         hoverinfo='text',
                         mode='lines+markers',
                         marker = dict(size=list(map(SetNewsSize, df_casos['noticia'])),
                                       color=['orange']*df_casos.shape[0])
                         )
             )

fig.show()

In [ ]:
df_casos.loc[df_casos['noticia'].notna()]